# Импорт библиотек и загрузка данных

In [ ]:
import pandas as pd
import numpy as np

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation
import spacy
from langdetect import detect
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

import tensorflow as tf
import tensorflow_addons as tfa

C:\Users\User\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
nltk.download('stopwords')

nlp_eng = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
nlp_rus = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])

stop_words_rus = stopwords.words('russian')
stop_words_eng = stopwords.words('english')
stop_words = stop_words_rus+stop_words_eng+['шт', 'каждый день', 'каждый', 'день', 'красная цена', 'красная', 'цена', 'верный', 'дикси', 'моя', 'моя цена', 'окей','то, что надо!', 'smart','spar', 'ашан']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка токенизатора и модели

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
with open('LSTM_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [ ]:
loaded_model = load_model("LSTM_model.h5", custom_objects={'Addons>F1Score': tfa.metrics.F1Score})

# Создание функций

## Загрузка данных

In [ ]:
def get_fake_transactions(df):

    return df[df['client'] == 'Шариков Алексей Юрьевич'].to_dict(orient='records')

## Обработка текстов покупок

In [ ]:
def preprocess_sentences(sentences, tokenizer, max_length):
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    return padded_sequences

## Подготовка текста для модели

In [ ]:
 def tokenize_text(products):

        all_sentence = []

        for value in products:
            try:
                lang = detect(value)

                if lang == 'en':
                    doc = nlp_eng(value)
                    lemmas = [token.lemma_ for token in doc if token.is_alpha and token.text not in punctuation and token.text.lower() not in stop_words]
                    cleaned_sentence = " ".join(lemmas)
                    all_sentence.append(cleaned_sentence)
                else:
                    doc = nlp_rus(value)
                    lemmas = [token.lemma_ for token in doc if token.is_alpha and token.text not in punctuation and token.text.lower() not in stop_words]
                    cleaned_sentence = " ".join(lemmas)
                    all_sentence.append(cleaned_sentence)
            except Exception as e:
                print(f"Error processing value: {value}")
                print(f"Error message: {str(e)}")

        padded_sequences = preprocess_sentences(all_sentence, tokenizer, 29)

        return padded_sequences

# Подготовка и обзор данных

In [ ]:
df2 = pd.read_excel('final_df.xlsx', sheet_name='Sheet1')

In [ ]:
transactions = get_fake_transactions(df2)

Посмотрим на названия продуктов и их категории

In [ ]:
product_names = [transaction['sale'] for transaction in transactions]
true_topics = [transaction['topic'] for transaction in transactions]

In [ ]:
product_names

['Свеча зажигания ngk 7422 bpr5es 1 шт.',
 'Змз вкладыши коренные змз 2101 0.5 21010-1000102-22',
 'Теплообменник vag 7h0 819 032',
 'Dennerle humin elixier средство для подготовки водопроводной воды, 500 мл',
 'Dennerle carbo elixier bio удобрение для растений, 500 мл',
 'Жевательный мармелад фру-фру трубочка тропические фрукты 35...',
 'Эскалоп из индейки пава-пава охлажденный 600 г',
 'Пряники полет шоколадные 250 г',
 'Биойогурт актибио с черносливом 2,9% 130 г',
 'Жевательная резинка orbit сочный арбуз без сахара 13,6 г',
 'Колбаса варено-копченая ближние горки сервелат австрийский...',
 'Батончики рот фронт с вафельной крошкой',
 'Плитка milka молочная 85 г',
 'Мармелад fruittella yo!rms жевательный 138 г',
 'Творог зерненый савушкин с клубникой 5% бзмж 130 г',
 'Кондиционер для детского белья vernel 1,82 л',
 'Гель glorix med 7 в 1 дезинфицирующий 750 мл',
 'Хозяйственное мыло аист концентрированное с глицерином 150 г',
 'Ватные палочки ola! silk sense 200 шт',
 'Порошок заводъ 

In [ ]:
true_topics

['автозапчасти',
 'автозапчасти',
 'автозапчасти',
 'аквариум',
 'аквариум',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'уборка',
 'уборка',
 'уборка',
 'уборка',
 'уборка',
 'уборка',
 'аквариум',
 'аквариум',
 'аквариум',
 'аквариум',
 'закуски и приправы',
 'закуски и приправы',
 'закуски и приправы',
 'напитки',
 'напитки']

Обработка текстов

In [ ]:
tokens = tokenize_text(product_names)

# Предсказание категорий и сравнение с оригиналом

In [ ]:
predictions = np.argmax(loaded_model.predict(tokens), axis=1)

1/1 [==============================] - 0s 90ms/step


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [ ]:
predictions = np.argmax(loaded_model.predict(tokens), axis=1)

1/1 [==============================] - 0s 40ms/step


In [ ]:
predictions

array([0, 1, 0, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 5,
       5, 5, 5, 4, 4, 4, 2, 3], dtype=int64)

In [ ]:
dictionary = { "topic": ['автозапчасти', 'видеоигры', 'напитки', 'продукты питания', 'закуски и приправы', 'аквариум', 'одежда', 'уборка', 'электроника', 'образование'], "label": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] }
topics=[]
for index in range(len(predictions)):
  label = predictions[index].item()
  topic = dictionary['topic'][dictionary['label'].index(label)]
  topics.append(topic)


In [ ]:
topics

['автозапчасти',
 'видеоигры',
 'автозапчасти',
 'аквариум',
 'аквариум',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'продукты питания',
 'уборка',
 'уборка',
 'уборка',
 'уборка',
 'уборка',
 'уборка',
 'аквариум',
 'аквариум',
 'аквариум',
 'аквариум',
 'закуски и приправы',
 'закуски и приправы',
 'закуски и приправы',
 'напитки',
 'продукты питания']

Посмотрим на точность

In [ ]:
correct_predictions = sum(predicted == true for predicted, true in zip(topics, true_topics))
print(f'Точность прогноза: {((correct_predictions / len(true_topics)) * 100):.2f}%')

Точность прогноза: 93.33%


Очень высокая точность, свыше 93 %

Посмотрим, где конкретно ошиблась модель

In [ ]:
incorrect_indices = [i for i, (predicted, true) in enumerate(zip(topics, true_topics)) if predicted != true]

In [ ]:
for i in incorrect_indices:
    print(f"Продукт: {product_names[i]}")
    print(f"Истинная метка: {true_topics[i]}")
    print(f"Предсказанная метка: {topics[i]}")
    print("--------------")

Продукт: Змз вкладыши коренные змз 2101 0.5 21010-1000102-22
Истинная метка: автозапчасти
Предсказанная метка: видеоигры
--------------
Продукт: Тараллини nina farina классические 180 г
Истинная метка: напитки
Предсказанная метка: продукты питания
--------------


Как видим, одна "ошибка", это и вовсе исправление ошибки из разметки!